In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import os

In [47]:
amz_books = pd.read_csv('books_data_with_index.csv')

In [3]:
amz_books.head()

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount,index
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN,0
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN,1
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN,2
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN,3
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN,4


In [3]:
# number of images expected (add counter to check all images have been pulled)
amz_books['image'].unique().shape[0]

149388

In [4]:
amz_books.shape

(212404, 11)

In [5]:
amz_books = amz_books.dropna(how="any", subset=["image"])

In [6]:
# n.b. 160,329 non Nan values, but I don't think these are all unique, so might need to pull a subset again 
# or exclude certain ids
amz_books.shape

(160329, 11)

In [7]:
amz_books.isnull().sum()

Title                 1
description       23372
authors            5302
image                 0
previewLink           0
publisher         30146
publishedDate       924
infoLink              0
categories         8445
ratingsCount     113202
index                 0
dtype: int64

In [ ]:
# ran this code to get all images
# computer went to sleep and only got upto index 137,962
folder_path = 'images'

no_images_saved = 0
no_images_not_saved = 0

for index, row in amz_books.iterrows():
    url = row["image"]
    image_url = str(url) + ".jpg"

    # Send a GET request to the image URL
    image_response = requests.get(image_url)

    # Check if the request was successful
    if image_response.status_code == 200:
        
        # Save the image to the specified folder
        save_name = os.path.join(folder_path, str(row["index"]) + ".jpg")
        image = Image.open(BytesIO(image_response.content))
        image.save(save_name)
        no_images_saved += 1
    else:
        no_images_not_saved += 1

In [ ]:
no_images_saved

In [ ]:
no_images_not_saved

In [8]:
# create slice of df to get rest of images
# going to test getting desired slice first!

data = pd.DataFrame()
data['index'] = [0, 1, 2, 3, 4, 5]
data['name'] = ["leanne", "joe", "lucien", "craig", "grando", "tiggy"]
data

,index,name
0,0,leanne
1,1,joe
2,2,lucien
3,3,craig
4,4,grando
5,5,tiggy


In [12]:
data_slice = data[3:]
data_slice

,index,name
3,3,craig
4,4,grando
5,5,tiggy


In [13]:
# so for my slice it will be df[137963:]
# issue - although this is the last file in my folder, it probaby isn't the highest number
amz_books_slice = amz_books[137963:] 

In [46]:
amz_books_slice['index'].min()

179556

In [17]:
# get all the images into a list, strip .jpg and find max value
# order df by index and then get slice
directory = 'images'
image_name_list = os.listdir(directory)

In [18]:
image_name_list[:10]

['110369.jpg',
 '14147.jpg',
 '86425.jpg',
 '52112.jpg',
 '108526.jpg',
 '89716.jpg',
 '63.jpg',
 '107615.jpg',
 '6400.jpg',
 '42301.jpg']

In [19]:
char = '.jpg'
image_list_strip_jpg = [ele.replace(char, '') for ele in image_name_list]

In [25]:
image_list_strip_jpg[:10]

['110369',
 '14147',
 '86425',
 '52112',
 '108526',
 '89716',
 '63',
 '107615',
 '6400',
 '42301']

In [26]:
len(image_list_strip_jpg)

105957

In [23]:
# seems there's a error because one of the strings is not a valid integer
# not sure why this would be 
image_list_strip_jpg_int = [int(x) for x in image_list_strip_jpg]

ValueError: invalid literal for int() with base 10: '.ipynb_checkpoints'

In [27]:
# filter out not valid int
valid_strings = [x for x in image_list_strip_jpg if x.isdigit()]
image_list_strip_jpg_int = [int(x) for x in valid_strings]


In [28]:
image_list_strip_jpg[0:10]

['110369',
 '14147',
 '86425',
 '52112',
 '108526',
 '89716',
 '63',
 '107615',
 '6400',
 '42301']

In [29]:
len(image_list_strip_jpg_int)

105956

In [30]:
105957-105956

1

In [33]:
# have to hope this isn't the min no. in my slice!
type(image_list_strip_jpg_int[0])

int

In [37]:
# so 137962 is the max value!
max(image_list_strip_jpg_int)

137962

In [ ]:
# maybe should reload df, take slice and then remove missing values

In [48]:
amz_books = pd.read_csv('books_data_with_index.csv')

In [49]:
amz_books_slice = amz_books[137963:] 

In [52]:
amz_books_slice['index'].min()

137963

In [53]:
amz_books_slice['index'].max()

212403

In [54]:
# that's better!
amz_books_slice.shape

(74441, 11)

In [55]:
amz_books_slice.isnull().sum()

Title                0
description      25490
authors          13293
image            20068
previewLink      10870
publisher        28097
publishedDate    11314
infoLink         10870
categories       16477
ratingsCount     57292
index                0
dtype: int64

In [56]:
amz_books_slice = amz_books_slice.dropna(how="any", subset=["image"])

In [57]:
amz_books_slice.isnull().sum()

Title                0
description       7763
authors           1699
image                0
previewLink          0
publisher        10104
publishedDate      287
infoLink             0
categories        2784
ratingsCount     38053
index                0
dtype: int64

In [58]:
# now can get remaining images
folder_path = 'images'

no_images_saved = 0
no_images_not_saved = 0

for index, row in amz_books_slice.iterrows():
    url = row["image"]
    image_url = str(url) + ".jpg"

    # Send a GET request to the image URL
    image_response = requests.get(image_url)

    # Check if the request was successful
    if image_response.status_code == 200:
        
        # Save the image to the specified folder
        save_name = os.path.join(folder_path, str(row["index"]) + ".jpg")
        image = Image.open(BytesIO(image_response.content))
        image.save(save_name)
        no_images_saved += 1
    else:
        no_images_not_saved += 1

In [59]:
no_images_saved

54373

In [60]:
no_images_not_saved

0